In [31]:
%reset -f

In [32]:
!nvidia-smi

Wed Apr  5 16:24:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 T...    On | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8               11W / 125W|   3030MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [33]:
!conda env export > "/home/ning/Data/Dropbox/Working_Directory/trial_chute/yolo_vino_tsrt.yml"

In [35]:
from ultralytics import YOLO
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [38]:
!rm -rf ./runs/detect/

data_path = "/home/ning/Data/Dropbox/Working_Directory/trial_chute/datasets/dataset.yaml"

In [39]:
model = YOLO("yolov8s.yaml")
model = YOLO("yolov8s.pt")  

results_v8n = model.train(data=data_path, epochs=100, dropout=True, seed=1)

results_v8n = model.val() 

model.info(verbose=True)  


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.Conv                  [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.C2f                   [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.Conv                  [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.C2f                   [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.Conv                  [256

In [44]:
model.export(format="onnx", dynamic=True, half=False)

Ultralytics YOLOv8.0.61 🚀 Python-3.10.10 torch-1.13.0+cu117 CPU

PyTorch: starting from runs/detect/train/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (21.4 MB)

ONNX: starting export with onnx 1.13.1...
ONNX: export success ✅ 0.4s, saved as runs/detect/train/weights/best.onnx (42.5 MB)

Export complete (0.6s)
Results saved to /home/ning/Data/Dropbox/Working_Directory/trial_chute/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=/home/ning/Data/Dropbox/Working_Directory/trial_chute/datasets/dataset.yaml 
Visualize:       https://netron.app


'runs/detect/train/weights/best.onnx'

In [45]:
model.export(format="openvino", dynamic=True, half=False)

Ultralytics YOLOv8.0.61 🚀 Python-3.10.10 torch-1.13.0+cu117 CPU

PyTorch: starting from runs/detect/train/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (21.4 MB)

ONNX: starting export with onnx 1.13.1...
ONNX: export success ✅ 0.7s, saved as runs/detect/train/weights/best.onnx (42.5 MB)

OpenVINO: starting export with openvino 2022.3.0-9052-9752fafe8eb-releases/2022/3...
OpenVINO: export success ✅ 0.5s, saved as runs/detect/train/weights/best_openvino_model/ (42.7 MB)

Export complete (1.4s)
Results saved to /home/ning/Data/Dropbox/Working_Directory/trial_chute/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best_openvino_model imgsz=640 
Validate:        yolo val task=detect model=runs/detect/train/weights/best_openvino_model imgsz=640 data=/home/ning/Data/Dropbox/Working_Directory/trial_chute/datasets/dataset.yaml 
Visualize:       https://netron.app


'runs/detect/train/weights/best_openvino_model'

In [46]:
!mkdir ./models/small_640_vino/
!cp ./runs/detect/train/weights/best_openvino_model/* ./models/small_640_vino/
!cp ./runs/detect/train/weights/best.pt ./models/small_640.pt
!cp ./runs/detect/train/weights/best.onnx ./models/small_640.onnx

mkdir: cannot create directory ‘./models/small_640_vino/’: File exists


In [47]:
model = YOLO("/home/ning/Data/Dropbox/Working_Directory/trial_chute/models/small_640.pt")  
results = model.predict(source='/home/ning/Data/Dropbox/Working_Directory/trial_chute/test/*.jpg', save=True, save_txt=True)


image 1/698 /home/ning/Data/Dropbox/Working_Directory/trial_chute/test/iroccv-t12-raisechute-labelling-20230330-022741543624_v0_00000.jpg: 384x640 1 chute, 1 safety_handrail, 27.6ms
image 2/698 /home/ning/Data/Dropbox/Working_Directory/trial_chute/test/iroccv-t12-raisechute-labelling-20230330-022741543624_v0_00005.jpg: 384x640 1 chute, 1 safety_handrail, 26.9ms
image 3/698 /home/ning/Data/Dropbox/Working_Directory/trial_chute/test/iroccv-t12-raisechute-labelling-20230330-022741543624_v0_00010.jpg: 384x640 1 chute, 1 safety_handrail, 26.9ms
image 4/698 /home/ning/Data/Dropbox/Working_Directory/trial_chute/test/iroccv-t12-raisechute-labelling-20230330-022741543624_v0_00015.jpg: 384x640 1 chute, 1 safety_handrail, 26.9ms
image 5/698 /home/ning/Data/Dropbox/Working_Directory/trial_chute/test/iroccv-t12-raisechute-labelling-20230330-022741543624_v0_00020.jpg: 384x640 1 chute, 1 safety_handrail, 27.0ms
image 6/698 /home/ning/Data/Dropbox/Working_Directory/trial_chute/test/iroccv-t12-raisech

In [49]:
import cv2
import numpy as np
import glob
 
img_array = []
jpg_order = sorted(glob.glob('/home/ning/Data/Dropbox/Working_Directory/trial_chute/runs/detect/predict/*.jpg'))

for filename in jpg_order:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('./pred/blind_test_small_640.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
model = YOLO("yolov8n.yaml")
model = YOLO("yolov8n.pt")  

results_v8n = model.train(data=data_path, epochs=100, dropout=True, seed=1, imgsz=1280)

results_v8n = model.val() 

model.info(verbose=True)  